# Análisis de sentimientos de vuelos realizados en 2015 mediante una base de datos extraída de tweeter.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats 
import re
import string

In [2]:
data = pd.read_csv('C:/Users/Juan Carlos/Desktop/apuntes/Tweets.csv')

In [3]:
def Cambio_nombres (df):
    """This function recives a Data Frame and
    change the names of columns and their position 
    the Data Frame """
    #We make a copy of the Dataframe.
    data_modify = data.copy()
    #We change the names of columns.
    data_modify.columns = ['Tweet_id', 'Sentimiento_aerolinea', 'Grado_de_confianza',
               'Razon_negativa', 'Confianza_razon_negativa', 'Aerolinea', 
               'Aerolinea_sentimiento_oro', 'Nombre_usuario', 'Razon_negativa_oro', 
               'Retweets', 'Comentario', 'Tweet_coord', 'Fecha_creación_tweet', 
               'Región_tweet', 'Zona horaria']
    #We change the position of columns in the Dataframe, 
    order_columns = ['Tweet_id', 'Aerolinea', 'Nombre_usuario',
              'Sentimiento_aerolinea', 'Grado_de_confianza',
               'Razon_negativa', 'Confianza_razon_negativa',  
               'Aerolinea_sentimiento_oro',  'Razon_negativa_oro', 
               'Retweets', 'Comentario', 'Tweet_coord', 'Fecha_creación_tweet', 
               'Región_tweet', 'Zona horaria']
    data_modify = data_modify[order_columns]
    return data_modify
data_2 = Cambio_nombres(data)

In [4]:
data_2 = Cambio_nombres(data)
data_2

Tweet_id       Aerolinea   Nombre_usuario  \
0      570306133677760513  Virgin America          cairdin   
1      570301130888122368  Virgin America         jnardino   
2      570301083672813571  Virgin America       yvonnalynn   
3      570301031407624196  Virgin America         jnardino   
4      570300817074462722  Virgin America         jnardino   
...                   ...             ...              ...   
14635  569587686496825344        American  KristenReenders   
14636  569587371693355008        American         itsropes   
14637  569587242672398336        American         sanyabun   
14638  569587188687634433        American       SraJackson   
14639  569587140490866689        American        daviddtwu   

      Sentimiento_aerolinea  Grado_de_confianza          Razon_negativa  \
0                   neutral              1.0000                     NaN   
1                  positive              0.3486                     NaN   
2                   neutral              0.6837                     NaN   
3                  negative              1.0000              Bad Flight   
4                  negative              1.0000              Can't Tell   
...                     ...                 ...                     ...   
14635              positive              0.3487                     NaN   
14636              negative              1.0000  Customer Service Issue   
14637               neutral              1.0000                     NaN   
14638              negative              1.0000  Customer Service Issue   
14639               neutral              0.6771                     NaN   

       Confianza_razon_negativa Aerolinea_sentimiento_oro Razon_negativa_oro  \
0                           NaN                       NaN                NaN   
1                        0.0000                       NaN                NaN   
2                           NaN                       NaN                NaN   
3                        0.7033                       NaN                NaN   
4                        1.0000                       NaN                NaN   
...                         ...                       ...                ...   
14635                    0.0000                       NaN                NaN   
14636                    1.0000                       NaN                NaN   
14637                       NaN                       NaN                NaN   
14638                    0.6659                       NaN                NaN   
14639                    0.0000                       NaN                NaN   

       Retweets                                         Comentario  \
0             0                @VirginAmerica What @dhepburn said.   
1             0  @VirginAmerica plus you've added commercials t...   
2             0  @VirginAmerica I didn't today... Must mean I n...   
3             0  @VirginAmerica it's really aggressive to blast...   
4             0  @VirginAmerica and it's a really big bad thing...   
...         ...                                                ...   
14635         0  @AmericanAir thank you we got on a different f...   
14636         0  @AmericanAir leaving over 20 minutes Late Flig...   
14637         0  @AmericanAir Please bring American Airlines to...   
14638         0  @AmericanAir you have my money, you change my ...   
14639         0  @AmericanAir we have 8 ppl so we need 2 know h...   

      Tweet_coord       Fecha_creación_tweet   Región_tweet  \
0             NaN  2015-02-24 11:35:52 -0800            NaN   
1             NaN  2015-02-24 11:15:59 -0800            NaN   
2             NaN  2015-02-24 11:15:48 -0800      Lets Play   
3             NaN  2015-02-24 11:15:36 -0800            NaN   
4             NaN  2015-02-24 11:14:45 -0800            NaN   
...           ...                        ...            ...   
14635         NaN  2015-02-22 12:01:01 -0800            NaN   
14636         NaN  2015-02-22 11:59:46 -0800          Texas   
14

## Con esta función eliminamos las columnas que tuvieran valores que fuesen mayores a 5500 nulos y agregamos dos columnas más

In [5]:
def Agregar_columnas(df):
    #We make a copy of the Dataframe thah we will use. 
    data_modify = data_2.copy()
    #We create an other function to apply the condition to each column of the Dataframe.
    def clasificacion (grado : str) -> str :
        """
        Receives a value and evaluates it to a string.
        Return
        INT
        """
        if grado <= .20:
            return 'Muy baja'
        elif grado <= .40:
            return 'Baja'
        elif grado <= .60:
            return 'Moderada'
        elif grado <=.80:
            return 'Alta'
        else:
            return 'Muy alta'
    #We add the two columns we need
    data_modify.insert(5,'Clasificacion_grado_de_confianza', data_modify['Grado_de_confianza'].apply(clasificacion))
    data_modify.insert(8,'Clasificacion_razon_negativa', data_modify['Confianza_razon_negativa'].apply(clasificacion))
    #We eliminate the columns that have more than 5500 nulls values.  
    nulls_cols = data_modify.isnull().sum()
    nulls_cols = nulls_cols[nulls_cols > 0].sort_values()
    cols_eliminate = list(nulls_cols[nulls_cols > 5500].index)
    data_cleaning = data_modify.drop(cols_eliminate, axis = 1)
    #In the columns that still have nulls values we writw the "None" caption. 
    data_cleaning[['Razon_negativa', 'Confianza_razon_negativa', 'Región_tweet', 'Zona horaria' ]] = data_cleaning[['Razon_negativa', 'Confianza_razon_negativa', 'Región_tweet', 'Zona horaria' ]].fillna('None')
    return(data_cleaning)


In [6]:
Data_3 = Agregar_columnas(data_2)
Data_3

Tweet_id       Aerolinea   Nombre_usuario  \
0      570306133677760513  Virgin America          cairdin   
1      570301130888122368  Virgin America         jnardino   
2      570301083672813571  Virgin America       yvonnalynn   
3      570301031407624196  Virgin America         jnardino   
4      570300817074462722  Virgin America         jnardino   
...                   ...             ...              ...   
14635  569587686496825344        American  KristenReenders   
14636  569587371693355008        American         itsropes   
14637  569587242672398336        American         sanyabun   
14638  569587188687634433        American       SraJackson   
14639  569587140490866689        American        daviddtwu   

      Sentimiento_aerolinea  Grado_de_confianza  \
0                   neutral              1.0000   
1                  positive              0.3486   
2                   neutral              0.6837   
3                  negative              1.0000   
4                  negative              1.0000   
...                     ...                 ...   
14635              positive              0.3487   
14636              negative              1.0000   
14637               neutral              1.0000   
14638              negative              1.0000   
14639               neutral              0.6771   

      Clasificacion_grado_de_confianza          Razon_negativa  \
0                             Muy alta                    None   
1                                 Baja                    None   
2                                 Alta                    None   
3                             Muy alta              Bad Flight   
4                             Muy alta              Can't Tell   
...                                ...                     ...   
14635                             Baja                    None   
14636                         Muy alta  Customer Service Issue   
14637                         Muy alta                    None   
14638                         Muy alta  Customer Service Issue   
14639                             Alta                    None   

      Confianza_razon_negativa Clasificacion_razon_negativa  Retweets  \
0                         None                     Muy alta         0   
1                          0.0                     Muy baja         0   
2                         None                     Muy alta         0   
3                       0.7033                         Alta         0   
4                          1.0                     Muy alta         0   
...                        ...                          ...       ...   
14635                      0.0                     Muy baja         0   
14636                      1.0                     Muy alta         0   
14637                     None                     Muy alta         0   
14638                   0.6659                         Alta         0   
14639                      0.0                     Muy baja         0   

                                              Comentario  \
0                    @VirginAmerica What @dhepburn said.   
1      @VirginAmerica plus you've added commercials t...   
2      @VirginAmerica I didn't today... Must mean I n...   
3      @VirginAmerica it's really aggressive to blast...   
4      @VirginAmerica and it's a really big bad thing...   
...                                                  ...   
14635  @AmericanAir thank you we got on a different f...   
14636  @AmericanAir leaving over 20 minutes Late Flig...   
14637  @AmericanAir Please bring American Airlines to...   
14638  @AmericanAir you have my money, you change my ...   
14639  @AmericanAir we have 8 ppl so we need 2 know h...   

            Fecha_creación_tweet   Región_tweet                Zona horaria  
0      2015-02-24 11:35:52 -0800           None  Eastern Time (US & Canada)  
1      2015-02-24 11:15:59 -0800           None  Pacific Time (US & Canada)  
2      2015-02-24 11:15:48 -0800  

## Con esta función se cambian los índices del Dataframe y se limpia el texto de la columna "Comentarios".

In [7]:
def Limpieza_texto(df): 
    #We create a copy of the Dataframe
    data_modify = Data_3.copy()
    #We change the index of the Dataframe. 
    data_cleaning = data_modify.set_index('Tweet_id')
    #We add a function that helpus to eliminate some caracters of the Comentario column.
    def clean_text(text):
        '''Make text uppercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
        text = text.upper()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('[% s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text
    #We create a new clean column named "Comentario_limpio" with the help of Comentario column and is added in the tenth position.
    data_cleaning.insert(10,'Comentario_limpio', data_cleaning['Comentario'].apply(clean_text))
    #We eliminate the Comentario Column. 
    del(data_cleaning['Comentario'])
    return data_cleaning
    
    

In [8]:
Data_4 = Limpieza_texto(Data_3)
Data_4

Aerolinea   Nombre_usuario Sentimiento_aerolinea  \
Tweet_id                                                                    
570306133677760513  Virgin America          cairdin               neutral   
570301130888122368  Virgin America         jnardino              positive   
570301083672813571  Virgin America       yvonnalynn               neutral   
570301031407624196  Virgin America         jnardino              negative   
570300817074462722  Virgin America         jnardino              negative   
...                            ...              ...                   ...   
569587686496825344        American  KristenReenders              positive   
569587371693355008        American         itsropes              negative   
569587242672398336        American         sanyabun               neutral   
569587188687634433        American       SraJackson              negative   
569587140490866689        American        daviddtwu               neutral   

                    Grado_de_confianza Clasificacion_grado_de_confianza  \
Tweet_id                                                                  
570306133677760513              1.0000                         Muy alta   
570301130888122368              0.3486                             Baja   
570301083672813571              0.6837                             Alta   
570301031407624196              1.0000                         Muy alta   
570300817074462722              1.0000                         Muy alta   
...                                ...                              ...   
569587686496825344              0.3487                             Baja   
569587371693355008              1.0000                         Muy alta   
569587242672398336              1.0000                         Muy alta   
569587188687634433              1.0000                         Muy alta   
569587140490866689              0.6771                             Alta   

                            Razon_negativa Confianza_razon_negativa  \
Tweet_id                                                              
570306133677760513                    None                     None   
570301130888122368                    None                      0.0   
570301083672813571                    None                     None   
570301031407624196              Bad Flight                   0.7033   
570300817074462722              Can't Tell                      1.0   
...                                    ...                      ...   
569587686496825344                    None                      0.0   
569587371693355008  Customer Service Issue                      1.0   
569587242672398336                    None                     None   
569587188687634433  Customer Service Issue                   0.6659   
569587140490866689                    None                      0.0   

                   Clasificacion_razon_negativa  Retweets  \
Tweet_id                                                    
570306133677760513                     Muy alta         0   
570301130888122368                     Muy baja         0   
570301083672813571                     Muy alta         0   
570301031407624196                         Alta         0   
570300817074462722                     Muy alta         0   
...                                         ...       ...   
569587686496825344                     Muy baja         0   
569587371693355008                     Muy alta         0   
569587242672398336                     Muy alta         0   
569587188687634433                         Alta         0   
569587140490866689                     Muy baja         0   

                                                    Comentario_limpio  \
Tweet_id                                                                
570306133677760513                   VIRGINAMERICA WHAT DHEPBURN SAID   
570301130888122368  VIRGINAMERICA PLUS YOUVE ADDED COMMERCIALS TO ...   
570301083672813571  VIRGINAMERICA I D